Checking Torch and Cuda

In [7]:
import torch
print('CUDA Available:', torch.cuda.is_available())

CUDA Available: True


Defining paths

In [3]:
file_name = "sample-new-fidelity-acnt-stmt"
pdf_data_path = "./pdf_data/" + file_name + ".pdf"
text_data_path = "./text_data/" + file_name + ".txt"

First,  we want to convert our PDF to image and run it through an OCR model. I will use Tesseract OCR here

In [1]:
import pdfplumber
import pytesseract
def extract_text_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        text = ''
        for page in pdf.pages:
            pil_image = page.to_image().original
            text += pytesseract.image_to_string(pil_image, config='--psm 6')
    return text

In [6]:
import os

pdf_text = ''
os.makedirs("text_data", exist_ok=True)
if not os.path.exists(f"text_data/{file_name}.txt"):
    with open(f"text_data/{file_name}.txt", "w") as f:
        pdf_text = extract_text_from_pdf(pdf_data_path)
        f.write(pdf_text)

Now that we have our text-file, we want to transform our query into something better based on the info in the document.